In [58]:
def bp_fit(X, target, layer_conf, max_epoch, max_error=.1, learn_rate=.1, print_per_epoch=100):
    np.random.seed(1)
    nin = [np.empty(i) for i in layer_conf]
    n = [np.empty(j + 1) if i < len(layer_conf) - 1 else np.empty(j) for i, j in enumerate(layer_conf)]
    w = np.array([np.random.rand(layer_conf[i] + 1, layer_conf[i + 1]) for i in range(len(layer_conf) - 1)])
    dw = [np.empty((layer_conf[i] + 1, layer_conf[i + 1])) for i in range(len(layer_conf) - 1)]
    d = [np.empty(s) for s in layer_conf[1:]]
    din = [np.empty(s) for s in layer_conf[1:-1]]
    epoch = 0
    mse = 1

    for i in range(0, len(n)-1):
        n[i][-1] = 1
    
    while (max_epoch == -1 or epoch < max_epoch) and mse > max_error:
        epoch += 1
        mse = 0
        
        for r in range(len(X)):
            n[0][:-1] = X[r]
            
            for L in range(1, len(layer_conf)):
                nin[L] = np.dot(n[L-1], w[L-1])
                n[L][:len(nin[L])] = sig(nin[L])
            e = target[r] - n[-1]
            mse += sum(e ** 2)
            d[-1] = e * sigd(nin[-1])
            dw[-1] = learn_rate * d[-1] * n[-2].reshape((-1, 1))
            
            for L in range(len(layer_conf) - 1, 1, -1):
                din[L-2] = np.dot(d[L-1],np.transpose(w[L-1][:-1]))
                d[L-2] = din[L-2] * np.array(sigd(nin[L-1]))
                dw[L-2] = (learn_rate * d[L-2]) * n[L-2].reshape((-1, 1))
            
            w += dw
        
        mse /= len(X)
        
        if print_per_epoch > -1 and epoch % print_per_epoch == 0: print(f'Epoch {epoch}, MSE: {mse}')
    return w, epoch, mse

In [59]:
def bp_predict(X, w):
    n = [np.empty(len(i)) for i in w]
    nin = [np.empty(len(i[0])) for i in w]
    predict = []
    
    n.append(np.empty(len(w[-1][0])))
    
    for x in X:
        n[0][:-1] = x
        
        for L in range(0, len(w)):
            nin[L] = np.dot(n[L], w[L])
            n[L + 1][:len(nin[L])] = sig(nin[L])
    
        predict.append(n[-1].copy())
    
    return predict

In [60]:
def sig(X):
  return [1 / (1 + np.exp(-x)) for x in X]
def sigd(X):
  output = []
  for i, x in enumerate(X):
    s = sig([x])[0]
    output.append(s * (1 - s))
  return output

In [61]:
import numpy as np
def onehot_enc(lbl, min_val=0):
    mi = min(lbl)
    enc = np.full((len(lbl), max(lbl) - mi + 1), min_val, np.int8)
    for i, x in enumerate(lbl):
        enc[i, x - mi] = 1
    return enc
    
def onehot_dec(enc, mi=0):
    return [np.argmax(e) + mi for e in enc]

In [71]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score
import pandas as pd

iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3,random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf=(4, 3, 3), learn_rate=.1, max_epoch=10000, max_error=.001, print_per_epoch=100)

print(f'Epochs: {ep}, MSE: {mse}')

predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)

print('Output:', predict)
print('True  :', y_test)
print('Accuracy:', accuracy)

<ipython-input-58-3f04843ee786>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  w = np.array([np.random.rand(layer_conf[i] + 1, layer_conf[i + 1]) for i in range(len(layer_conf) - 1)])
<ipython-input-58-3f04843ee786>:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  w += dw


Epoch 100, MSE: 0.19045841193641894
Epoch 200, MSE: 0.07191039575015529
Epoch 300, MSE: 0.05477329557645173
Epoch 400, MSE: 0.04868920708563229
Epoch 500, MSE: 0.04552486755428171
Epoch 600, MSE: 0.04362905088225506
Epoch 700, MSE: 0.04241256560334685
Epoch 800, MSE: 0.041594650366039744
Epoch 900, MSE: 0.04102273137630642
Epoch 1000, MSE: 0.0406083359660748
Epoch 1100, MSE: 0.04029801289043292
Epoch 1200, MSE: 0.04005841386726616
Epoch 1300, MSE: 0.03986814230065838
Epoch 1400, MSE: 0.03971311802789583
Epoch 1500, MSE: 0.03958385779556943
Epoch 1600, MSE: 0.03947383561024381
Epoch 1700, MSE: 0.03937846948604513
Epoch 1800, MSE: 0.03929448104045514
Epoch 1900, MSE: 0.039219482316550806
Epoch 2000, MSE: 0.03915170409681103
Epoch 2100, MSE: 0.03908981406378582
Epoch 2200, MSE: 0.03903279303963392
Epoch 2300, MSE: 0.03897984938618559
Epoch 2400, MSE: 0.03893035885568421
Epoch 2500, MSE: 0.0388838216480035
Epoch 2600, MSE: 0.03883983124208961
Epoch 2700, MSE: 0.038798051366308726
Epoch 280

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3,random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf=(4, 2, 3), learn_rate=.1, max_epoch=100, max_error=.5, print_per_epoch=25)

print(f'Epochs: {ep}, MSE: {mse}')

predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)

print('Output:', predict)
print('True  :', y_test)
print('Accuracy:', accuracy)

<ipython-input-58-3f04843ee786>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  w = np.array([np.random.rand(layer_conf[i] + 1, layer_conf[i + 1]) for i in range(len(layer_conf) - 1)])
<ipython-input-58-3f04843ee786>:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  w += dw


Epoch 25, MSE: 0.5587148548510854
Epochs: 29, MSE: 0.4956900384526158
Output: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
True  : [0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2, 1]
Accuracy: 0.28888888888888886


In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3,random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf=(4, 25, 3), learn_rate=.1, max_epoch=10000, max_error=.01, print_per_epoch=25)

print(f'Epochs: {ep}, MSE: {mse}')

predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)

print('Output:', predict)
print('True  :', y_test)
print('Accuracy:', accuracy)

<ipython-input-58-3f04843ee786>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  w = np.array([np.random.rand(layer_conf[i] + 1, layer_conf[i + 1]) for i in range(len(layer_conf) - 1)])
<ipython-input-58-3f04843ee786>:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  w += dw


Epoch 25, MSE: 1.9997475602526067
Epoch 50, MSE: 1.9996839107720699
Epoch 75, MSE: 1.9995729256252905
Epoch 100, MSE: 1.9993234166160632
Epoch 125, MSE: 1.998004842894972
Epoch 150, MSE: 1.5034310711672474
Epoch 175, MSE: 0.9882734055300789
Epoch 200, MSE: 0.9358385629366136
Epoch 225, MSE: 0.8962403847767554
Epoch 250, MSE: 0.5720113088187334
Epoch 275, MSE: 0.17189641558830474
Epoch 300, MSE: 0.13551945448567232
Epoch 325, MSE: 0.10976277201695424
Epoch 350, MSE: 0.0915915464816905
Epoch 375, MSE: 0.07929291362007071
Epoch 400, MSE: 0.07095077966556532
Epoch 425, MSE: 0.0651161047205166
Epoch 450, MSE: 0.060880862438032445
Epoch 475, MSE: 0.057698280991125867
Epoch 500, MSE: 0.05523313404391773
Epoch 525, MSE: 0.05327321999520484
Epoch 550, MSE: 0.05168001906066302
Epoch 575, MSE: 0.050360624096748914
Epoch 600, MSE: 0.04925114278017032
Epoch 625, MSE: 0.04830653743243756
Epoch 650, MSE: 0.047494247901885826
Epoch 675, MSE: 0.0467901200965703
Epoch 700, MSE: 0.04617577407555434
Epoch